# Улучшаем векторизаторы с помощью библиотек NLTK и pymorphy

В этом тюториале мы научимся векторизовывать тексты с использованием стемминга и лемматизации.

Импортируем модули которые нам понадобятся впоследствии:

In [23]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk import stem
import pymorphy3 as pymorphy

Предположим, что на входе нам дана коллекция из нескольких текстовых документов (названия институтов):

In [24]:
texts = [
    "Московская государственная академия хореографии",
    "Московский государственный университет им. М.В. Ломоносова (Университет МГУ)",
    "Московский физико-технический институт (национальный исследовательский университет)",
    "Национальный исследовательский университет «МИЭТ»",
    "Национальный исследовательский университет ИТМО",
]
print(texts)

['Московская государственная академия хореографии', 'Московский государственный университет им. М.В. Ломоносова (Университет МГУ)', 'Московский физико-технический институт (национальный исследовательский университет)', 'Национальный исследовательский университет «МИЭТ»', 'Национальный исследовательский университет ИТМО']


## TF-IDF-векторизаторы с поддержкой стемминга

Создадим кастомный _TfidfVectorizer_ с поддержкой стемминга.

Для этого нам потребуется унаследоваться от соответствующего векторизатора из библиотеки _scikit-learn_ и переопределить метод _build_analyzer_:

In [25]:
class StemTfidfVectorizer(TfidfVectorizer):
    def __init__(self, lang='english', **kwargs):
        super().__init__(**kwargs)
        self.lang = lang

    def build_analyzer(self):
        """Called only once so we don't need to cache stemmer"""
        analyzer = super().build_analyzer()
        stemmer = stem.SnowballStemmer(self.lang)
        return lambda text: (stemmer.stem(word) for word in analyzer(text))

Обратите внимание, что:
- мы передаем в конструктор в качестве параметра язык, для которого мы хотим создать стеммер
- в качестве стеммера мы будем использовать _SnowballStemmer_ из библиотеки _NLTK_, однако с тем же успехом можно было бы использовать и любой другой стеммер
- мы можем создавать объект стеммера непосредственно в методе _build_analyzer_ т.к. он вызывается ровно один раз во время создания объекта векторизатора, т.е. можно не опасаться что наш стеммер будет создаваться на каждый вызов _fit_ (а создание стеммера это потенциально медленная операция)

Также заметим, что переопределение метода _build_analyzer_ -- это не единственный возможный способ научить векторизаторы стеммингу. Класс _TfidfVectorizer_ поддерживает альтернативный способ кастомизации через параметр конструктора _tokenizer=_, детали можно найти в документации: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Попробуем воспользоваться нашим векторизатором, и сравним результаты его работы с "обычным" векторизатором:

In [26]:
# Create and fit vanilla vectorizer
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(texts)

# Create and fit custom vectorizer
stem_vectorizer = StemTfidfVectorizer(lang='russian', min_df=1)
X_stem = stem_vectorizer.fit_transform(texts)

print(X.shape)
print(X_stem.shape)

(5, 17)
(5, 15)


Сразу видно, что в результате применения векторизатора со стеммингом получилась матрица меньшего размера.

Распечатаем теперь термины-фичи:

In [27]:
print(vectorizer.get_feature_names_out())
print(stem_vectorizer.get_feature_names_out())

['академия' 'государственная' 'государственный' 'им' 'институт'
 'исследовательский' 'итмо' 'ломоносова' 'мгу' 'миэт' 'московская'
 'московский' 'национальный' 'технический' 'университет' 'физико'
 'хореографии']
['академ' 'государствен' 'им' 'институт' 'исследовательск' 'итм'
 'ломоносов' 'мгу' 'миэт' 'московск' 'национальн' 'техническ'
 'университет' 'физик' 'хореограф']


Видим, что векторизатор с поддержкой стемминга действительно оперирует не словами, а основами слов (русскоязычный эквивалент английского _stem_), и наше снижение размерности было достигнуто за счет того, что:
- слова 'государственная' и 'государственный' склеились в стем 'государствен'
- слова 'московская' и 'московский' склеились в терм 'московск'

Дальше мы можем использовать наш _StemTfidfVectorizer_ по полной аналогии с ванильным, в т.ч. для векторизации текстов и ранжирования по формуле TF-IDF.

## TF-IDF-векторизаторы с поддержкой лемматизации

По полной аналогии с предыдущим примером, создадим кастомный _TfidfVectorizer_ с поддержкой лемматизации.

Саму лемматизацию будем производить с помощью библиотеки _pymorphy_:

In [28]:
def get_best_normal_form(morph, word):
    """Helper function: applies analyzer to word and extracts best normal form"""
    parsed = morph.parse(word)
    if not parsed:
        raise RuntimeError(f"cant parse word '{word}'")
    best_form = parsed[0]
    return best_form.normal_form

class MorphTfidfVectorizer(TfidfVectorizer):
    def __init__(self, lang='ru', **kwargs):
        super().__init__(**kwargs)
        self.lang = lang

    def build_analyzer(self):
        """Called only once so we don't need to cache analyzer"""
        analyzer = super().build_analyzer()
        morph = pymorphy.MorphAnalyzer(lang=self.lang)
        return lambda text: (get_best_normal_form(morph, word) for word in analyzer(text))

И теперь применим _MorphTfidfVectorizer_ к нашей коллекции текстов:

In [29]:
morph_vectorizer = MorphTfidfVectorizer(lang='ru', min_df=1)
X_morph = morph_vectorizer.fit_transform(texts)
print(X_morph.shape)
print(morph_vectorizer.get_feature_names_out())

(5, 15)
['академия' 'государственный' 'институт' 'исследовательский' 'итмый'
 'ломоносов' 'мгу' 'миэта' 'московский' 'национальный' 'они' 'технический'
 'университет' 'физико' 'хореография']


Видим, что новый векторизатор работает уже на уровне лемм, а не основ слов.

Любопытно, что слова 'государственная' и 'государственный' опять склеились, но этот раз уже к правильной лемме 'государственный'. Аналогичная ситуация и со словами  'московская' и 'московский'.

Также, бросаются слова несколько странностей лемматизации, например слово "итмо" было лемматизировано как "итмый", а "миэт" -- в лемму "миэта". Такие кейсы, к сожалению, являются типичными проблемами лемматизации, и с ними обычно борятся с помощью различных словарей слов-исключений, содержащих списки имен, географических названий и т.п.

Тем не менее, в прикладных задачах лемматизация как правило работает гораздо лучше, чем стемминг. Это верно, в первую очередь, для языков с "богатой" морфологией, таких как русский.

Дальше мы можем использовать наш _MorphTfidfVectorizer_ по полной аналогии с ванильным, в т.ч. для векторизации текстов и ранжирования по формуле TF-IDF.